In [ ]:
import os
import time
import ollama

base_dir = '.'  # current folder

llm_dir = os.path.join(base_dir, "llm_obf")
trad_obf_dir = os.path.join(base_dir, "trad_obf", "obf")

code_data = {}

def build_prompt(js_obf_jsobf, js_obf_llm, js_original):
    prompt = f"""
Can you try to deobfuscate this obfuscated JavaScript code using a traditional JS obfuscator:
```js
{js_obf_jsobf} And this obfuscated JavaScript code generated by an LLM: {js_obf_llm} And compare each of them to the original unobfuscated JavaScript code:
{js_original} Please provide a detailed comparison of the percentage of similarity for each obfuscated version relative to the original, focusing on:

    Functionality

    Structure

    Naming

    Comments

    Overall similarity

Also, explain key differences in the obfuscation techniques used by the JS obfuscator and the LLM.
"""
    return prompt.strip()

model = ollama.chat(model='mistral', options={"num_gpu_layers": 0})
for filename in os.listdir(base_dir):
    if filename.endswith('.js'):
        try:
           base_path = os.path.join(base_dir, filename)
           llm_path = os.path.join(llm_dir, filename)
           trad_path = os.path.join(trad_obf_dir, f"output-{filename}")
           # Read original
           with open(base_path, "r", encoding="utf-8") as f:
            original = f.read()
            # Read LLM-obfuscated
            try:
                with open(llm_path, "r", encoding="utf-8") as f:
                    llm_obf = f.read()
            except FileNotFoundError:
                llm_obf = None

            # Read traditional obfuscated
            try:
                with open(trad_path, "r", encoding="utf-8") as f:
                    trad_obf = f.read()
            except FileNotFoundError:
                trad_obf = None

            prompt = build_prompt(trad_obf, llm_obf, original)
            print("File:" + filename)
            res = ollama.chat(
                model='mistral',
                messages=[{'role': 'user', 'content': prompt}],
                options={"num_gpu_layers": 0}
            )

            content = res['message']['content']

            print(content)
        except Exception as e:
            print(f"\n❌ Error processing {filename}: {e}")

prompt = "Based on all the previous requests, provide a summary of the differences between both types of obfuscation and an overall summary of the comparisons."
res = ollama.chat(
                model='mistral',
                messages=[{'role': 'user', 'content': prompt}],
                options={"num_gpu_layers": 0}
            )
print(res)

File:02_background_click.js
1. Functionality: Both obfuscated versions have identical functionality as they both perform the same action of changing the background color of the document body to a random hexadecimal color. The original, obfuscated using a traditional JavaScript (JS) obfuscator, and the one generated by an LLM (Language Model) all have 100% similar functionality.

2. Structure: The structure of both obfuscated versions is quite different from the original. The JS obfuscator uses variable names that are encoded using an array of predefined strings, which makes it harder to read and understand the code without decoding it. In contrast, the LLM-generated version has a simpler structure and resembles the original more closely, but still with some differences in naming and syntax. The similarity in terms of structure is approximately 50% between the obfuscated JS version and the original, while it's around 80% for the LLM-generated version.

3. Naming: In the traditional JS o